<a href="https://colab.research.google.com/github/Anuragpandey2005/ola/blob/main/My_olaproject_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd

In [ ]:
df= pd.read_csv("https://d2beiqkhq929f0.cloudfront.net/public_assets/assets/000/002/492/original/ola_driver_scaler.csv")

In [ ]:
df.head()

In [ ]:
df["MMM-YY"]=pd.to_datetime(df["MMM-YY"], errors='coerce')

In [ ]:
df

In [ ]:
df.isnull().sum()/len(df) * 100

In [ ]:
df.nunique()

In [ ]:
df.info()

In [ ]:
df.describe()

In [ ]:
df.isna().sum()

In [ ]:
mean=df["Age"].mean()
df["Age"]= df["Age"].fillna(mean)

In [ ]:
mode=df["Gender"].mode()
df["Gender"]=df["Gender"].fillna(mode[0])
mode

In [ ]:
df.isna().sum()

In [ ]:
import numpy as np

df["Target"] = np.where(df["LastWorkingDate"].notna(), 1, 0)

In [ ]:
df

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
sns.scatterplot(data=df,x="Income",y="Age",hue = "Target")

* low income is related to driver churn , it can be observed from scaterplot


In [ ]:
# Convert 'Dateofjoining' and 'LastWorkingDate' to datetime objects
df["Dateofjoining"] = pd.to_datetime(df["Dateofjoining"], errors='coerce', format='%d/%m/%y')
df["LastWorkingDate"] = pd.to_datetime(df["LastWorkingDate"], errors='coerce', format='%d/%m/%y')

# This will result in a Timedelta object
df["Tenure"] = df["LastWorkingDate"] - df["Dateofjoining"]

In [ ]:
df

In [ ]:
df[df["Total Business Value"]==0]

In [ ]:
mean_income=df["Income"].mean()
mean_income

In [ ]:
median_income =df["Income"].median()
median_income

In [ ]:
mode_income= df["Income"].mode()
mode_income

In [ ]:
plt.figure(figsize=(12, 6))
sns.countplot(data=df, x="City", hue="Target", palette="Set2")
plt.xticks(rotation=45, ha='right')
plt.title("Comparison of Target 0 and 1 Counts per City")
plt.ylabel("Number of Drivers")
plt.xlabel("City")
plt.tight_layout()
plt.show()

* c- 20 has more driver churn then average of other city combined

In [ ]:
sns.countplot(data=df,x="Target")

* driver who does not left job is very much compare to who left so it is highly imbalanced

In [ ]:
df

In [ ]:
df.groupby("Driver_ID")["MMM-YY"].value_counts()

In [ ]:
df_grouped=df.groupby("Education_Level")["Income"].median()

In [ ]:
df_grouped

* drivers with higher education_level have better income

In [ ]:
df_grouped1=df.groupby("Education_Level")["Target"].sum()

In [ ]:
df_grouped1

* their is no corellation between education level and driver attrition

In [ ]:
dokha = df[df["Total Business Value"] < 0].groupby("Target").size()


In [ ]:
dokha

In [ ]:
dokha_sum = df[df["Total Business Value"] < 0].groupby("Target")[["Total Business Value"]].sum()

*  output suggest the their is high probability the driver with negative business value greater than 4 lakh will not leave the job.
* output also suggest the their is high probability the driver with negative business value less than 2.5 lakh will  leave the job.

In [ ]:
df_grouped_mean_income=df.groupby("Target")["Income"].mean()
df_grouped_mean_income

In [ ]:
df_grouped_median_income=df.groupby("Target")["Income"].median()
df_grouped_median_income

* The driver with income less the 10000 of avrrage income of ola driver have very high probability to leave the their job
* the company need to focus on driver with income less than 10000 of average income on ola drivers.


In [ ]:
sns.lineplot(data=df,x="Age",y="Income",hue="Target")

:

# Key Insights from the Chart
1. Overall Trend
Income increases with age for both churned and retained drivers until mid-40s.

There's a slight drop beyond age ~50, likely due to fewer drivers or semi-retirement.

2. Churned vs Not Churned
Non-churned drivers (Target = 0) consistently earn higher income than churned drivers across most age groups.

The gap is especially visible from age 30 to 50.

Churned drivers (Target = 1) have a flatter income growth curve — suggesting lower income may be a churn signal.

3. Income Stability
Non-churned group shows tighter shaded bands (lower variance) ⇒ more stable income.

Churned group shows wider bands ⇒ income instability, which may contribute to churn.

4. Sharp Spikes and Drops
The spike at age ~55 for non-churned might be an outlier (few drivers with very high income).

Drop after age 55 could be due to small sample size — worth flagging

In [ ]:
df

In [ ]:
df["LastWorkingDate"].dt.year.notna()

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
df

In [ ]:
df["LastWorkingDate"].fillna(0)

In [ ]:
end_date = pd.to_datetime("2020-12-31")
# Calculate tenure for those who haven't left (where LastWorkingDate is NaN)
# and fill the NaN values in the 'Tenure' column
df["Tenure"] = df["Tenure"].fillna(end_date - df["Dateofjoining"])

In [ ]:
df["LastWorkingDate"]= df["LastWorkingDate"].fillna(0)

In [ ]:
df

In [ ]:
from sklearn.preprocessing import OneHotEncoder

In [ ]:
df_encoded = pd.get_dummies(df, columns=["City","Education_Level"])

In [ ]:
# Initialize the OneHotEncoder
df=df_encoded

In [ ]:
df=df.drop(["MMM-YY","Dateofjoining","LastWorkingDate"],axis=1)

In [ ]:
df

In [ ]:
# Convert 'Tenure' to numerical (number of days)
df['Tenure'] = df['Tenure'].dt.days
# Features and target
x = df[["Age", "Gender", "Income", "Joining Designation", "Grade", "Total Business Value", "Quarterly Rating", "Tenure"]]
y = df["Target"]


In [ ]:
from imblearn.over_sampling import SMOTE

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=42)


In [ ]:
smote = SMOTE(random_state=42)
x_train_resampled, y_train_resampled = smote.fit_resample(x_train, y_train)

In [ ]:
from sklearn.linear_model import LogisticRegression

In [ ]:
model= LogisticRegression()

In [ ]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.metrics import classification_report
categorical_cols = ["Gender", "Grade"]
numerical_cols = ["Age", "Income", "Joining Designation", "Total Business Value", "Quarterly Rating", "Tenure"]

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=42)

# Preprocessor: One-hot encode categorical columns
preprocessor = ColumnTransformer(
    transformers=[
        ("cat", OneHotEncoder(drop="first"), categorical_cols)
    ],
    remainder="passthrough"  # keep numerical columns as-is
)
model = LogisticRegression()
model.fit(x_train, y_train)
y_pred = model.predict(x_test)
print(classification_report(y_test, y_pred))


In [ ]:
model.fit(x_train_resampled, y_train_resampled)

# Evaluate
y_pred = model.predict(x_test)
print(classification_report(y_test, y_pred))


In [ ]:
from sklearn.metrics import f1_score

# Predict on test set
y_pred = model.predict(x_test)

# Calculate F1 Score
f1 = f1_score(y_test, y_pred, average='binary')  # use 'macro', 'micro', or 'weighted' for multiclass
print("F1 Score:", f1)


In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
model= RandomForestClassifier(random_state = 42)

In [ ]:
model.fit(x_train_resampled, y_train_resampled)

In [ ]:
y_pred = model.predict(x_test)
print(classification_report(y_test, y_pred))

y_pred = model.predict(x_test)

# Calculate F1 Score
f1 = f1_score(y_test, y_pred, average='binary')  # use 'macro', 'micro', or 'weighted' for multiclass
print("F1 Score:", f1)

In [ ]:
!pip install xgboost

In [ ]:
from xgboost import XGBClassifier

In [ ]:
model=XGBClassifier()

In [ ]:
model.fit(x_train_resampled, y_train_resampled)

In [ ]:
y_pred = model.predict(x_test)
print(classification_report(y_test, y_pred))

y_pred = model.predict(x_test)

# Calculate F1 Score
f1 = f1_score(y_test, y_pred, average='binary')  # use 'macro', 'micro', or 'weighted' for multiclass
print("F1 Score:", f1)

In [ ]:
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, roc_auc_score


# Train Logistic Regression
log_model = LogisticRegression(max_iter=1000)
log_model.fit(x_train_resampled, y_train_resampled)
log_proba = log_model.predict_proba(x_test)[:, 1]

# Train Random Forest
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(x_train_resampled, y_train_resampled)
rf_proba = rf_model.predict_proba(x_test)[:, 1]

# Train XGBoost
xgb_model = XGBClassifier(use_label_encoder=False, eval_metric='logloss')
xgb_model.fit(x_train_resampled, y_train_resampled)
xgb_proba = xgb_model.predict_proba(x_test)[:, 1]

# Compute FPR, TPR for each model
fpr_log, tpr_log, _ = roc_curve(y_test, log_proba)
fpr_rf, tpr_rf, _ = roc_curve(y_test, rf_proba)
fpr_xgb, tpr_xgb, _ = roc_curve(y_test, xgb_proba)

# AUC Scores
auc_log = roc_auc_score(y_test, log_proba)
auc_rf = roc_auc_score(y_test, rf_proba)
auc_xgb = roc_auc_score(y_test, xgb_proba)

# Plot
plt.figure(figsize=(10, 7))
plt.plot(fpr_log, tpr_log, label=f"Logistic (AUC = {auc_log:.2f})")
plt.plot(fpr_rf, tpr_rf, label=f"Random Forest (AUC = {auc_rf:.2f})")
plt.plot(fpr_xgb, tpr_xgb, label=f"XGBoost (AUC = {auc_xgb:.2f})")
plt.plot([0, 1], [0, 1], 'k--')  # diagonal
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("ROC Curve Comparison")
plt.legend()
plt.grid(True)
plt.show()
